#### **Gruppo 5.b**: Caponi Marco (matricola: 508773) - Ceneda Gianluca (matricola: 488257)

# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: coordintervals


Variabili utili per testare il funzionamento


In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads
using Test
using DataStructures


In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di coordintervals

coordintervals crea un dizionario ordinato dove la chiave è l'intervallo su una coordinata, e come valore associato ha l'indice dell'intervallo corrispondente nel boundig box

In [3]:
function coordintervals(coord,bboxes)
	boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
	for (h,box) in enumerate(bboxes)
		key = box[coord,:]
		if haskey(boxdict,key) == false
			boxdict[key] = [h]
		else
			push!(boxdict[key], h)
		end
	end
	return boxdict
end

coordintervals (generic function with 1 method)

In [4]:
@btime coordintervals(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])   #0.000014 seconds

  954.850 ns (16 allocations: 1.19 KiB)


OrderedDict{Vector{Float64}, Vector{Int64}} with 2 entries:
  [0.0, 1.0] => [1]
  [0.0, 0.0] => [2]

### controllo se la funzione è type unstable

La funzione **non** è type unstable in quanto ritorna una stringa.

Body: :OrderedDict{Array{Float64,1},Array{Int64,1}}

### versione modificata coordintervals

Questa funzione è molto semplice non sono serviti grandi interventi. La condizione dell'if è diventata !haskey(boxdict,key) da haskey(boxdict,key) == false

In [6]:
function coordintervalsMOD(coord,bboxes)
     boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
     l = length(bboxes)
@simd     for h=1:l
@views         key = bboxes[h][coord,:]
         if !haskey(boxdict,key)
             boxdict[key] = [h]
         else 
             push!(boxdict[key], h)
         end
     end
     return boxdict
 end
 

coordintervalsMOD (generic function with 1 method)

In [7]:
@btime coordintervalsMOD(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])   #0.000014 seconds

  1.023 μs (16 allocations: 1.19 KiB)


OrderedDict{Vector{Float64}, Vector{Int64}} with 2 entries:
  [0.0, 1.0] => [1]
  [0.0, 0.0] => [2]

## TEST

In [11]:
using Test

@testset "coordintervals Tests" begin
	# 2x2x2 cuboidal grid for 1-, 2-, and 3-dim tests
	V,(VV,EV,FV,CV) = Lar.cuboidGrid([2,2,2],true)
	W,_ = Lar.apply(Lar.r(1,1,pi/6),(V,[VV,EV,FV,CV]))
			
		cellpoints = [ W[:,EV[k]]::Lar.Points for k=1:length(EV) ]
		bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
		@testset "Edge tests" begin # 
			@test typeof(Lar.coordintervals(1,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(2,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(3,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
		end
		cellpoints = [ W[:,FV[k]]::Lar.Points for k=1:length(FV) ]
		bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
		@testset "Face tests" begin # 
			@test typeof(Lar.coordintervals(1,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(2,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(3,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
		end
		cellpoints = [ W[:,CV[k]]::Lar.Points for k=1:length(CV) ]
		bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
		@testset "Cell tests" begin # 
			@test typeof(Lar.coordintervals(1,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(2,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
			@test typeof(Lar.coordintervals(3,bboxes)) == 
				OrderedDict{Array{Float64,1}, Array{Int64,1}}
		end
	end

Test Summary:        | Pass  Total
coordintervals Tests |    9      9


Test.DefaultTestSet("coordintervals Tests", Any[Test.DefaultTestSet("Edge tests", Any[], 3, false, false), Test.DefaultTestSet("Face tests", Any[], 3, false, false), Test.DefaultTestSet("Cell tests", Any[], 3, false, false)], 0, false, false)

![test di coordintervals](https://github.com/MarcoCap13/LARSplitting2D/blob/main/docs/test/coordintervals_test.png?raw=true)

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [9]:
@benchmark coordintervals(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])

BenchmarkTools.Trial: 10000 samples with 19 evaluations.
 Range (min … max):  950.526 ns … 481.607 μs  ┊ GC (min … max):  0.00% … 99.56%
 Time  (median):       1.007 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.280 μs ±   8.452 μs  ┊ GC (mean ± σ):  13.11% ±  1.99%

  ██▇▅▅▄▃▂▁▁                                                    ▂
  ███████████▆▆▆▆▅▅▅▄▅▆▅▄▄▅▆▆▇▇▇██▇▇▆▇▆▅▄▅▅▅▅▄▅▄▆▄▄▄▃▅▄▄▅▂▄▄▃▃▄ █
  951 ns        Histogram: log(frequency) by time       3.07 μs <

 Memory estimate: 1.19 KiB, allocs estimate: 16.

funzione modificata:

In [10]:
@benchmark coordintervalsMOD(1, [[0.0 1.0; 0.0 0.0], [0.0 0.0; 0.0 0.1]])

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.028 μs …  2.407 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.149 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.238 μs ± 29.876 μs  ┊ GC (mean ± σ):  8.83% ± 1.41%

  █▇▅▃▂▁  ▁▂▃▄▄▂▁                                            ▂
  ████████████████▇▇▆▅▅▅▅▅▆▅▆▆▆▄▆▅▅▃▄▅▅▅▅▅▃▃▅▄▃▄▁▄▃▃▁▃▃▄▅▅▁▄ █
  1.03 μs      Histogram: log(frequency) by time     7.05 μs <

 Memory estimate: 1.19 KiB, allocs estimate: 16.